In [1]:
import xarray as xr
import numpy as np 
import pandas as pd
from hython.metrics import compute_fdc_fms, compute_fdc_fhv, compute_fdc_flv, compute_hr, compute_far, compute_csi

from hython.utils import write_to_zarr, read_from_zarr, set_seed



#### Read Data and some pre-processing

In [2]:
SURROGATE_INPUT = "https://eurac-eo.s3.amazonaws.com/INTERTWIN/SURROGATE_INPUT/adg1km_eobs_original.zarr/"
train_temporal_range = slice("2012-01-01","2022-12-31")

target_names = ["q_river", "vwc"]

Y = (
    read_from_zarr(url=SURROGATE_INPUT , group="y")
    .sel(time=train_temporal_range)[target_names]
)

simulated_data = Y.q_river.sel(lat=45.4, lon=11, method="nearest")
simulated = simulated_data.to_dataframe().reset_index()
simulated = simulated.drop(columns=['lat', 'lon', 'layer'])
simulated['time'] = pd.to_datetime(simulated['time'])
simulated

start_date = '2012-01-01'
end_date = '2022-12-31'
data = pd.read_csv("/mnt/CEPH_PROJECTS/InterTwin/hydrologic_data/ADO/ADO_discharge.csv", parse_dates=['date'])
data = data.loc[data['id_station'] == "ADO_DSC_ITH3_0002"]
data['time'] = pd.to_datetime(data['date'])
data = data.drop(columns=['date', 'quality', 'id_station'])
observed = data[(data['time'] >= start_date) & (data['time'] <= end_date)].reset_index(drop=True)
observed

,discharge_m3_s,time
0,72.01,2012-01-01
1,74.25,2012-01-02
2,71.90,2012-01-03
3,72.97,2012-01-04
4,72.80,2012-01-05
...,...,...
4013,72.19,2022-12-27
4014,72.45,2022-12-28
4015,73.40,2022-12-29
4016,73.88,2022-12-30


#### Check FDC biases

In [6]:
biasFMS = compute_fdc_fms(observed, "discharge_m3_s", simulated, "q_river")
biasFHV = compute_fdc_fhv(observed, "discharge_m3_s", simulated, "q_river")
biasFLV = compute_fdc_flv(observed, "discharge_m3_s", simulated, "q_river")

BiasFMS : 11.29575404991648
BiasFHV : 101.067346123209
BiasFLV : -198.83580291790952


#### Now for the soil moisture metrics

In [13]:
import xarray as xr
import numpy as np

def create_random_dataset(original_ds: xr.Dataset) -> xr.Dataset:
    """
    Create a random xarray.Dataset based on the structure of an existing xarray.Dataset.
    
    Parameters:
    original_ds (xr.Dataset): The original dataset to copy structure from.
    
    Returns:
    xr.Dataset: A new dataset with the same dimensions and coordinates but with random data.
    """
    random_data_vars = {}
    
    # Loop through each variable in the original dataset
    for var_name, var_data in original_ds.data_vars.items():
        # Generate random data of the same shape as the original variable
        random_data = np.random.random(var_data.shape)
        
        # Create a new DataArray with the same coordinates and assign the random data
        random_data_vars[var_name] = xr.DataArray(
            data=random_data,
            dims=var_data.dims,
            coords=var_data.coords
        )
    
    # Create a new dataset with the same coordinates and dimensions
    random_ds = xr.Dataset(data_vars=random_data_vars, coords=original_ds.coords)
    
    return random_ds


randomised_data_array = create_random_dataset(Y).vwc
Y_data_array = Y.vwc

In [14]:
hit_rate = compute_hr(Y_data_array, randomised_data_array)
far = compute_far(Y_data_array, randomised_data_array)
csi = compute_csi(Y_data_array, randomised_data_array)

/home/sdhinakaran/micromamba/envs/hython/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1650: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


{'wet_threshold_0.8_hit_rate': 20.00161241595442, 'dry_threshold_0.2_hit_rate': 20.00515342555146}


/home/sdhinakaran/micromamba/envs/hython/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1650: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


{'wet_threshold_0.8_far': 92.24332374004418, 'dry_threshold_0.2_far': 92.27527968032285}


/home/sdhinakaran/micromamba/envs/hython/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1650: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


{'wet_threshold_0.8_csi': 5.920061370714957, 'dry_threshold_0.2_csi': 5.901736856063945}
